# Header
### Compare DX12 confidence mask to NPIPE confidence mask 

In [2]:
%%capture
import numpy as np
import pandas as pd
from healpy.sphtfunc import smoothing
import smica
import os
import json

import copy
from astropy.io import fits

import healpy as hp
import component_separation
from component_separation.io import IO
import component_separation.interface as cslib
import component_separation.MSC.MSC.pospace as ps
import component_separation.powspec as pw

from component_separation.cs_util import Config
from component_separation.cs_util import Constants as const
from component_separation.cs_util import Helperfunctions as hpf
from component_separation.cs_util import Planckf, Planckr, Plancks
import matplotlib.pyplot as plt
csu = Config()
io = IO(csu)
import warnings
warnings.filterwarnings('ignore')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
tebdict = {0:'I', 1:'Q', 2:'U'}

lmax=csu.lmax
nside_out = csu.nside_out
bins = csu.bins
detector = csu.PLANCKMAPFREQ_f
specfilter = csu.specfilter
freqfilter = csu.freqfilter
lmaxbins = int(bins[-1][1])
colors=csu.CB_color_cycle
filename = io.fh.total_filename
C_lS = io.load_powerspectra('signal')

In [33]:
cfDXS = copy.deepcopy(csu.cf)
cfDXS['pa']['freqdset'] = 'DX12'
cfDXS['pa']['mskset'] = 'smica'
csuDXS = Config(cfDXS)
ioDXS = IO(csuDXS)
tmask, pmask_smica, pmask_smica = ioDXS.load_one_mask_forallfreq()

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Run with the following settings:
{'Tscale': 'K_CMB', 'mskset': 'smica', 'freqdset': 'DX12', 'Spectrum_scale': 'C_l', 'lmax': 4000, 'lmax_mask': 6000, 'freqdatsplit': '', 'smoothing_window': 0, 'max_polynom': 0, 'num_sim': 5, 'binname': 'SMICA_highell_bins', 'overwrite_cache': True, 'split': '', 'nside_out': None, 'splitvariation': 'GHz', 'freqfilter': ['545', '857'], 'specfilter': ['TB', 'EB', 'ET', 'BT', 'BE'], 'nside_desc_map': [1024, 2048]}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
loading mask ['psmaskP_2048.fits.gz', 'gmaskP_apodized_0_2048.fits.gz']


AttributeError: 'dict' object has no attribute 'shape'

In [40]:
psmask = hp.read_map("/global/homes/s/sebibel/data/mask/psmaskP_2048.fits.gz")
gmask = hp.read_map("/global/homes/s/sebibel/data/mask/gmaskP_apodized_0_2048.fits.gz")

In [42]:
hp.mollzoom(psmask)
hp.mollzoom(gmask)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'DX12'
cfc['pa']['mskset'] = 'smica'
csuc = Config(cfc)
ioc = IO(csuc)
maps_DX = ioc.load_plamap(csuc.cf, field=(0,1,2), nside_out=csu.nside_out)

cfc = copy.deepcopy(csu.cf)
cfc['pa']['freqdset'] = 'NPIPE'
cfc['pa']['mskset'] = 'smica'
csuc = Config(cfc)
ioc = IO(csuc)
maps_NP = ioc.load_plamap(csuc.cf, field=(0,1,2), nside_out=csuc.nside_out)

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Run with the following settings:
{'Tscale': 'K_CMB', 'mskset': 'lens', 'freqdset': 'DX12', 'Spectrum_scale': 'C_l', 'lmax': 4000, 'lmax_mask': 6000, 'freqdatsplit': '', 'smoothing_window': 0, 'max_polynom': 0, 'num_sim': 5, 'binname': 'SMICA_highell_bins', 'overwrite_cache': True, 'split': '', 'nside_out': None, 'splitvariation': 'GHz', 'freqfilter': ['545', '857'], 'specfilter': ['TB', 'EB', 'ET', 'BT', 'BE'], 'nside_desc_map': [1024, 2048]}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
loaded {'030': '/global/cfs/cdirs/cmb/data/planck2018/pr3/frequencymaps/LFI_SkyMap_030_1024_R3.00_full.fits', '044': '/global/cfs/cdirs/cmb/data/planck2018/pr3/frequencymaps/LFI_SkyMap_044_1024_R3.00_full.fits', '070': '/global/cfs/cdirs/cmb/data/planck2018/pr3/frequencymaps/LFI_SkyMap_070_1024_R3.00_full.fits', '100': '/global/cfs/cdirs/cmb/data/planck2018/pr3/frequencymaps/HFI_SkyMap_100_2048_R3.01_full.fits', '143': '/global/cfs/cdirs/cmb/data/planck2018/pr3/frequ

# Bright Pixels

In [58]:
freqs = csu.PLANCKMAPFREQ_f
treshold = 10
print(freqs)
logi_NP = dict()
logi_DX = dict()
for idx, n in enumerate(freqs):
    maps_NPug = dict()
    maps_NPug[n]=np.zeros(shape=(3,maps_NP['100'][0].shape[0]))
    maps_DXug = dict()
    maps_DXug[n]=np.zeros(shape=(3,maps_DX['100'][0].shape[0]))
    logi_NP[idx] = dict()
    logi_DX[idx] = dict()
    print(n)
    for idy, m in enumerate(range(3)):
        maps_NPug[n][idy] = hp.ud_grade(maps_NP[n][idy],nside_out=2048)
        maps_DXug[n][idy] = hp.ud_grade(maps_DX[n][idy],nside_out=2048)
        logi_NP[idx][idy] = np.abs(maps_NPug[n][idy])>treshold*np.std(maps_NPug[n][idy])
        logi_DX[idx][idy] = np.abs(maps_DXug[n][idy])>treshold*np.std(maps_DXug[n][idy])
#     ret[n] = np.where(np.abs(data[n])>np.mean(data[n])+10*np.std(data[n]), 0.0, data[n])

['030', '044', '070', '100', '143', '217', '353']
030
044
070
100
143
217
353


In [63]:
stackedNP = np.zeros_like(logi_NP[3][1])
stackedDX = np.zeros_like(logi_DX[3][1])
for n in range(0,7):
    for m in range(1,3):
        stackedNP = np.logical_or(stacked, logi_NP[n][m])
        stackedDX = np.logical_or(stacked, logi_DX[n][m])

In [66]:
%matplotlib widget
hp.mollview(~stackedNP)
hp.mollzoom(stackedDX)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
print(np.sum(stackedNP), np.sum(stackedDX))
print(np.sum(np.logical_xor(stackedNP, stackedDX)))

22905 23942
1037


In [70]:
hp.mollzoom(np.logical_xor(stackedNP, stackedDX))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
hp.mask_bad(maps_DXug['100'][1])

KeyError: '100'

In [57]:
maps_DXug

{'353': array([ 2.84132344e-04, -2.30353544e-04,  2.61506240e-04, ...,
         1.91385407e-04,  8.31828038e-06, -3.82281723e-05])}

In [ ]:
hp.mollzoom(logi_NP[idf][idm], title = 'NPIPE')
hp.mollzoom(logi_DX[idf][idm], title = 'DX')

In [ ]:
for idf in range(7):
    for idm in range(3):
        print(logi_DX[idf][idm][logi_DX[idf][idm]==1].shape, logi_NP[idf][idm][logi_NP[idf][idm]==1].shape, freqs[idf] + ' '+ tebdict[idm])

In [ ]:
%matplotlib inline
for idf in [3,4,5]:
    for idm in range(1,3):
        print('Bright DX12:', logi_DX[idf][idm][logi_DX[idf][idm]==1].shape, '\nBright Npipe:', logi_NP[idf][idm][logi_NP[idf][idm]==1].shape)
        hp.mollzoom(logi_DX[idf][idm]+2*logi_NP[idf][idm], title= 'Bright pixels @ map '+ tebdict[idm] + ' w/ frequency ' + freqs[idf])
        plt.show()

In [ ]:
%matplotlib widget
for idf in [4]:
    for idm in [2]:
        print('Bright DX12:', logi_DX[idf][idm][logi_DX[idf][idm]==1].shape, '\nBright Npipe:', logi_NP[idf][idm][logi_NP[idf][idm]==1].shape)
        hp.mollzoom(logi_DX[idf][idm]+2*logi_NP[idf][idm], title= 'Bright pixels @ map '+ tebdict[idm] + ' w/ frequency ' + freqs[idf])

# Pointsources